# Imports and data loading

In [11]:

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

In [12]:
# loading only a sample for testing purposes
df = pd.read_csv('../raw_data/train.csv',nrows=300)

# Showing ground truth and prediction in text

In [112]:
#import custom.css into this notebook

def css():
    styles = open("./styles/custom.css", "r").read()
    return HTML('<style>'+styles+'</style>')
css()

In [104]:
def render_html(df):
    return "<{0} style='padding: 2px'>{1} <strong> [{0}] </strong></{0}>".format(df['discourse_type'],df['discourse_text'])

In [106]:
def comparison_text(prediction=pred, ground_truth=true):
    html = f"""
    <div class="content">
     <span style="font-size:16px">Legend --></span>
      <lead>Lead</lead>
      <Position>Position</Position>
      <Claim>Claim</Claim>
      <Counterclaim>Counterclaim</Counterclaim>
      <Rebuttal>Rebuttal</Rebuttal>
      <Evidence>Evidence</Evidence>
      <Concluding_Statement>Concluding_Statement</Concluding_Statement>
    </div>

    <div class="row">
      <div class="column">
        <h2 class="title">Prediction</h2>
        <p style="text-align:justify">{pred}</p>
      </div>
      <div class="column">
        <h2 class="title">Ground Truth</h2>
        <p style="text-align:justify">{true}</p>
      </div>
    </div>
    
    """
    
    return HTML(html)

In [102]:
# as if the output were post processed ; just to see 

df['html'] = df.apply(render_html, axis=1) #transform discourse_text in html txt with formating 
df_essays = df.groupby('id').agg({'html':' '.join,}).reset_index() #groupby essay id

true = df_essays.loc[7,'html'].replace('Concluding Statement','Concluding_Statement')
pred = df_essays.loc[15,'html'].replace('Concluding Statement','Concluding_Statement')

In [115]:
comparison_text(pred,true)